# VideoDB QuickStart 

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/quickstart/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VideoDB is a specialized database tailored for multimedia content.   
It allows you to store **videos as data**, enabling easy retrieval, streaming, and querying, much like how you would manage text data in a conventional database.

This notebook is designed to help you get started wlth VideoDB.   
For more information, please refer to the documentation available at [docs.videodb.io](https://docs.videodb.io)

<div style="height:40px;"></div>

## Setup
---  


### 🔧 Installing VideoDB in your environment

VideoDB is available as [python package 📦](https://pypi.org/project/videodb)  

In [ ]:
!pip install videodb

### 🔗 Setting Up a connection to db
To connect to VideoDB, simply create a `Connection` object. 

This can be done by either providing your VideoDB API key directly to the constructor or by setting the `VIDEO_DB_API_KEY` environment variable with your API key. 

> Your API key is available in the [VideoDB dashboard](https://console.videodb.io).

In [2]:
import videodb
import os

VIDEO_DB_API_KEY = os.environ.get("VIDEO_DB_API_KEY", "")
base_url = "https://api.videodb.io"
conn = videodb.connect(base_url=base_url, api_key=VIDEO_DB_API_KEY)

<div style="height:40px;"></div>

## Working with a single Video
---

<div style="height:10px;"></div>

### ⬆️ Uploading a video
Now that you have established a connection to VideoDB, you can now upload your videos using `Connection.upload()`.

You can upload a video to VideoDB in following ways
- By passing `url` to `Connection.upload` if video is available as a url
- By passing `file_path` to `Connection.upload` if its on your file system   

`Connection.upload` returns a `Video` Object, which can be used to access video


> VideoDB utilizes the concept of collections to organize videos.   
> By default, videos are uploaded to your default collection, but they can be separated into different collections for better organization.

In [3]:
#Upload a video by url
vid1 = conn.upload(url="https://www.youtube.com/watch?v=WDv4AWk0J3U")

#Upload a video from file system
#vid2 = conn.upload(file_path="./my_video.mp4")

<div style="height:15px;"></div>

### 📺 Viewing your video

Viewing your video is very simple. its instantly available and reading to use without the need of rendering ⚡️

- Generate a streamable url for video using `Video.generate_stream()`.
- Generate a playable url for video using `Video.play()` (you can open this url in browser).
- You can also embed this player in your product/website using an iframe. 

For this demo guide, we will embed VideoDB player in this notebook using `Ipython.display.IFrame`

<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
    <strong>Note:</strong>if you are viewing this notebook on github, you won't be able to see iframe player, because of security restrictions. <br> 
    Please open the printed link of player in your browser</div>


In [4]:
from IPython.display import IFrame 

player_url = vid1.play()
print(player_url)


player_width = 800
player_height = 400

IFrame(player_url, player_width, player_height)

None


<div style="height:15px;"></div>

### 🔍 Searching inside a video
 Searching inside a video is super easy with VideoDB  
  
  
**Index & Search inside Video** :    
  
You can **semantically** index a video and perform search operations. Use following function in sequence
- `Video.get_transcript()`  - gets a transcript for video  

- `Video.index_spoken_words()`  - Indexes the video semantically   
- `Video.search("Morning Sunlight")`  - Performs a search query on video  
     
  
    
    
**Viewing Search Results** :   
  
`Video.search()` will return a `SearchResults` Object, which contains the sections/shots of videos which semantically match your search query
- `SearchResults.get_shots()` - Returns a list of `Shot` that matched search query 

- `SearchResults.compile()` - Returns a stream link which a compilation of all shots. 
- `SearchResults.play()` - Returns a playable url for video (similar to `Video.play()` you can open this link in browser, or embed it into your website using iframe)

In [12]:
query = "Morning Sunlight"

## Index the video for Semantic Search
vid1.get_transcript()
vid1.index_spoken_words()

## Search and view results
search_results = vid1.search("Morning Sunlight")
compiled_result = search_results.compile()

## View the result in VideoDB Player
player_url = vid1.play() 
print(f"Compile Results for Query {query}", player_url)

Compile Results for Query Morning Sunlight None


<div style="height:15px;"></div>

### 🌟 More Methods on `Video` Object
There are mutiple methods available on a `Video` Object, that can be helpful for your usecase 

**Get Specific Sections of Videos:** :   
- `Video.generate_stream(timeline=[[0,10]])` : Clip specific sections of a video by passing `timeline`. 

**Add Subtitle to Video**  :
- `Video.add_subtitle()` : Adds subtitles to video. This returns a stream url. 

**Get Thumbnail of Video**  :
- `Video.generate_thumbnail()` : Returns a thumbnail image of video.

**Delete a video** :
- `Video.delete()` : Delete a video from Collection


In [ ]:
# Create a clip from 0 to 50s
vid1.generate_stream(timeline=[[0, 50]])
clip1_preview = vid1.play()

print("Clip Preview", clip1_preview)


# Add Subtitle
vid1.add_subtitle()
subtitle_preview = vid1.play()

# View Subtitled Video 
print("Subtitle Preview", subtitle_preview)

# Delete Video
vid1.delete()


<div style="height:40px;"></div>

## Working with Multiple Videos
---
VideoDB uses Collection to store Videos   
By default, videos are uploaded to your default collection, but they can be separated into different collections for better organization.

<div style="height:15px;"></div>

### 🔄 Using Collection to Fetch, Upload Mutliple Videos
- `Connection.get_collection()` : Returns `Collection` Object, the default collection 

- `Collection.upload()` : Returns `Video` Object, uploads the video to collection
- `Collection.get_videos()` : Returns list of `Video`, all videos in collections  
- `Collection.get_video(video_id)`: Returns `Video`, respective video object from give `video_id`   
- `Collection.delete_video(video_id)`: Deltes the video from Collection  

In [ ]:
# Get a collection
default_coll = conn.get_collection()

# Upload Videos to a collection
default_coll.upload(url="https://www.youtube.com/watch?v=lsODSDmY4CY")
default_coll.upload(url="https://www.youtube.com/watch?v=vZ4kOr38JhY")
default_coll.upload(url="https://www.youtube.com/watch?v=uak_dXHh6s4")
default_coll.upload(url="https://www.youtube.com/watch?v=S8nPJU9xkNw")

all_videos = default_coll.get_videos()
for video in all_videos:
    print(video.id, " : ", video.name)


<div style="height:15px;"></div>

### 📂 Search on Multiple Videos from a collection
- `Collection.search()` :  Returns `SearchResults`; Search on collection of videos

In [19]:
query = "Dopamine"

#Index all videos in collection
for video in default_coll.get_videos():
    video.get_transcript()
    video.index_spoken_words()


search_results =default_coll.search(query)
search_results.compile()
search_results_preview = search_results.play()

## View the result in VideoDB Player
print(f"Compile Results for Query {query}", search_results_preview)

Compile Results for Query Dopamine https://console.dev.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/a9d414f9-3bdb-4ba9-ace5-cdb6c586ade3.m3u8


<div style="height:40px;"></div>

## Further References
---
- You can find more tutorials here in this repository
- Checkout [videodb.io](https://videodb.io) for latest updates and features
